El conjunto de datos del archivo Hotel.csv contiene información de reserva para un hotel urbano y un hotel resort, e incluye información como cuándo se realizó la reserva, duración de la estadía, la cantidad de adultos, niños y / o bebés, y la cantidad de espacios de estacionamiento disponibles, entre otras cosas. Toda la información de identificación personal se ha eliminado de los datos. Ambos hoteles están ubicados en Portugal. La columna “is_canceled” (1 es sí, 0 es no) para saber si fue o no cancelada

In [9]:
import json 
import pandas as pd
import calendar
from calendar import monthrange
import pycountry

In [2]:
df = pd.read_json("data.json")
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,City Hotel,1,615,2017,March,11,16,0,2,2,...,Non Refund,1,NULL,0,Transient,62.0,0,0,Canceled,21/10/2015
1,City Hotel,1,615,2017,March,11,16,0,2,2,...,Non Refund,1,NULL,0,Transient,62.0,0,0,Canceled,21/10/2015
2,City Hotel,1,615,2017,March,11,16,0,2,2,...,Non Refund,1,NULL,0,Transient,62.0,0,0,Canceled,21/10/2015
3,City Hotel,1,615,2017,March,11,16,0,2,2,...,Non Refund,1,NULL,0,Transient,62.0,0,0,Canceled,21/10/2015
4,City Hotel,1,615,2017,March,11,16,0,2,2,...,Non Refund,1,NULL,0,Transient,62.0,0,0,Canceled,21/10/2015


In [3]:
df.shape

(119390, 32)

Limpieza de datos

Para realizar la limpieza de datos se eliminan los valores duplicados en caso de existir, así como posibles columnas y filas con valores vacíos. Por otra parte, en el caso del adr se toman en cuenta solo aquellos valores mayores o iguales a cero, asumiento que no se tiene la opción de que el hotel retribuya económicamente al cliente por su reservación. Debido a que se desconoce el contexto del hotel no se eliminan valores muy altos, pues podría ser una suite especial. En caso de que se obtenga más información acerca del hotel, se podrá delimitar un máximo para evitar tomar en cuenta posibles valores atípicos que perjudiquen los resultados. 

In [4]:
df = df.drop_duplicates()
df = df.dropna()
df = df.dropna(axis=1)
df = df[(df['adr'] >= 0)]
df.shape

(87395, 32)

A través de la limpieza delimitamos que se trabajará con 87396 reservaciones. 

¿De dónde vienen los huéspedes?

In [22]:
procedencia =  df['country'].value_counts()
print("Los huespedes provienen de los siguientes países: ", procedencia)


Los huespedes provienen de los siguientes países:  country
PRT    27453
GBR    10432
FRA     8837
ESP     7252
DEU     5387
       ...  
GLP        1
PYF        1
MDG        1
GUY        1
VGB        1
Name: count, Length: 178, dtype: int64


¿Cuánto pagan los huéspedes por una habitación por noche en promedio?

In [ ]:
promedio_por_noche = round(df['adr'].mean(),2)
print("El pago promedio por noche es: $", promedio_por_noche)

El pago promedio por noche es: $ 106.34


En ocasiones, el promedio puede verse afectado por valores poco comunes, por lo cual, una forma más confiable de definir cuánto pagan los huespedes de forma general por noche sería utilizando la mediana o la moda, pues, la primera se encuentra justo en medio de los valores totales y la segunda nos presenta los más frecuentes. En este caso se utiliza la mediana como segundo ejemplo para tener un mayor contexto de la cantidad de dinero que pagan los clientes por noche, disminuyendo el sesgo generado por una varianza alta. 

In [ ]:
mediana_por_noche = round(df['adr'].median(),2)
print("La mediana del pago por noche es: $", mediana_por_noche)

La mediana del pago por noche es: $ 98.1



¿Cómo varía el precio por noche durante el año?

In [ ]:
round(df['adr'].describe(),2)

count    87395.00
mean       106.34
std         55.01
min          0.00
25%         72.00
50%         98.10
75%        134.00
max       5400.00
Name: adr, dtype: float64

De forma general podemos observar que la desviación estandar del precio es de $55.01. Asimismo el 25% de los precios por noche se encuentran en el rango de $72.00 o menos. El 50% está en el rango de $98.10 o menos. Mientras que el 25% más alto está por encima de los $134.00. Asimismo, se observa que el máximo pagado fue de $5400.00,mientas que el mínimo es de $0.00 debido a que durante el proceso de limpieza de datos, los valores negativos no se tomaron en cuenta. 


¿Cuáles son los meses más ocupados?

In [24]:
ocupacion_meses = df['arrival_date_month'].value_counts()
print("Los meses más ocupados son:", ocupacion_meses[0:3])

Los meses más ocupados son: arrival_date_month
August    11257
July      10057
May        8355
Name: count, dtype: int64


In [25]:
ocupacion_meses

arrival_date_month
August       11257
July         10057
May           8355
April         7908
June          7765
March         7512
October       6934
September     6690
February      6098
December      5131
November      4995
January       4693
Name: count, dtype: int64

¿Cuánto tiempo se queda la gente en los hoteles (noches)?

In [ ]:
df['total_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']
avr_total_nights = round(df['total_nights'].mean())

print("En promedio la gente se queda por", avr_total_nights, "noches")

En promedio la gente se queda por 4 noches


Reservas por segmento de mercado

In [ ]:
reservas_segmento = df['market_segment'].value_counts()
print(reservas_segmento)

market_segment
Online TA        51618
Offline TA/TO    13889
Direct           11804
Groups            4941
Corporate         4212
Complementary      702
Aviation           227
Undefined            2
Name: count, dtype: int64


Los segmentos que más realizaron reservas fue el de Online TA, seguido por Offline TA/TO.

¿Cuántas reservas se cancelaron?

In [ ]:
cancelaciones = df['is_canceled'].value_counts()
print("Se cancelaron", cancelaciones[1], "reservas")

Se cancelaron 24025 reservas


¿Qué mes tiene el mayor número de cancelaciones?

In [ ]:
df.filter(items = ['arrival_date_month','is_canceled']).value_counts()[12:]

arrival_date_month  is_canceled
August              1              3623
July                1              3198
May                 1              2442
April               1              2409
June                1              2354
March               1              1830
October             1              1642
September           1              1642
February            1              1415
December            1              1378
November            1              1054
January             1              1038
Name: count, dtype: int64

El mes con mayor número de cancelaciones es Agosto, con 3623 cancelaciones.

Hacer tabla de correlación para las variables

In [ ]:
numeric_type = df.drop(['is_canceled'], axis=1).select_dtypes(include='number')
numeric_type.corr()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,total_nights
lead_time,1.000000,0.139120,0.101171,0.009885,0.235115,0.310112,0.140491,-0.003612,-0.147101,0.005376,-0.078949,0.076995,0.132154,0.023596,-0.086537,0.034253,0.318249
arrival_date_year,0.139120,1.000000,-0.514220,-0.010007,0.005106,0.003624,0.038616,-0.023333,0.024281,-0.054211,0.027242,0.008573,-0.027938,0.176121,-0.039803,0.064280,0.004599
arrival_date_week_number,0.101171,-0.514220,1.000000,0.093578,0.026910,0.027825,0.024365,0.014259,-0.036795,0.007197,-0.020805,0.011936,0.013842,0.098254,0.008942,0.046588,0.030726
arrival_date_day_of_month,0.009885,-0.010007,0.093578,1.000000,-0.017763,-0.028168,-0.001122,-0.000395,-0.004109,-0.008540,0.000167,0.006340,0.006587,0.022563,0.009163,-0.001662,-0.027565
stays_in_weekend_nights,0.235115,0.005106,0.026910,-0.017763,1.000000,0.555533,0.088282,0.013671,-0.107133,-0.020634,-0.056691,0.050241,-0.031679,0.039043,-0.042912,0.032426,0.786247
stays_in_week_nights,0.310112,0.003624,0.027825,-0.028168,0.555533,1.000000,0.095575,0.016011,-0.113654,-0.018780,-0.058524,0.085003,0.001906,0.053339,-0.044298,0.037839,0.950577
adults,0.140491,0.038616,0.024365,-0.001122,0.088282,0.095575,1.000000,0.016640,-0.171749,-0.042097,-0.120917,-0.048068,-0.015739,0.249010,0.007803,0.112775,0.103990
babies,-0.003612,-0.023333,0.014259,-0.000395,0.013671,0.016011,0.016640,1.000000,-0.012974,-0.005395,-0.009191,0.081040,-0.006840,0.023444,0.030884,0.094820,0.017003
is_repeated_guest,-0.147101,0.024281,-0.036795,-0.004109,-0.107133,-0.113654,-0.171749,-0.012974,1.000000,0.206408,0.441461,0.005913,-0.012813,-0.152949,0.072046,-0.001273,-0.124465
previous_cancellations,0.005376,-0.054211,0.007197,-0.008540,-0.020634,-0.018780,-0.042097,-0.005395,0.206408,1.000000,0.392068,-0.010267,0.003682,-0.050270,-0.003399,0.001870,-0.021661
